In [1]:
import os
import pandas as pd
import json
from tqdm import tqdm
from numpyencoder import NumpyEncoder

In [2]:
ROOT = os.path.join(os.getcwd(), 'Statsbomb_data\open-data-master')
DATA = os.path.join(ROOT, 'data')
EVENTS = os.path.join(DATA, 'events')
LINEUPS = os.path.join(DATA, 'lineups')
MATCHES = os.path.join(DATA, 'matches')
THREESIXTY = os.path.join(DATA, 'three-sixty')

# Competitions

In [42]:
competitions = pd.read_json(
    path_or_buf=os.path.join(DATA, 'competitions.json'),
    orient='records'
)
competitions.head()

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,9,27,Germany,1. Bundesliga,male,False,False,2015/2016,2023-08-17T23:51:11.837478,None,None,2023-08-17T23:51:11.837478
1,16,4,Europe,Champions League,male,False,False,2018/2019,2023-03-07T12:20:48.118250,2021-06-13T16:17:31.694,None,2023-03-07T12:20:48.118250
2,16,1,Europe,Champions League,male,False,False,2017/2018,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2021-01-23T21:55:30.425330
3,16,2,Europe,Champions League,male,False,False,2016/2017,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
4,16,27,Europe,Champions League,male,False,False,2015/2016,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00


In [43]:
print("UNIQUE COMPETITIONS:\n")
[f'{c}' for c in competitions.competition_name.unique()]

UNIQUE COMPETITIONS:



['1. Bundesliga',
 'Champions League',
 'Copa del Rey',
 "FA Women's Super League",
 'FIFA U20 World Cup',
 'FIFA World Cup',
 'Indian Super league',
 'La Liga',
 'Liga Profesional',
 'Ligue 1',
 'North American League',
 'NWSL',
 'Premier League',
 'Serie A',
 'UEFA Euro',
 'UEFA Europa League',
 "UEFA Women's Euro",
 "Women's World Cup"]

In [44]:
top_5_leagues = ['1. Bundesliga', 'La Liga', 'Premier League', 'Serie A', 'Ligue 1', 'Champions League']

In [45]:
competitions_f1 = competitions.loc[competitions.competition_name.isin(top_5_leagues)]
competitions_f1.replace(
    to_replace='1. Bundesliga',
    value='Bundesliga',
    inplace=True
)

C:\Users\HP\AppData\Local\Temp\ipykernel_15972\645922511.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  competitions_f1.replace(


In [46]:
print(f"FILTERED COMPETITIONS:\n")
[c for c in competitions_f1.competition_name.unique()]

FILTERED COMPETITIONS:



['Bundesliga',
 'Champions League',
 'La Liga',
 'Ligue 1',
 'Premier League',
 'Serie A']

In [47]:
print(f"UNIQUE GENDERS:\n")
[g for g in competitions_f1.competition_gender.unique()]

UNIQUE GENDERS:



['male']

In [48]:
print(f"UNIQUE COMPETITION YOUTH:\n")
[y for y in competitions_f1.competition_youth.unique()]

UNIQUE COMPETITION YOUTH:



[False]

In [49]:
print(f"SEASON RANGE:\n")
[season for season in competitions_f1.season_name.unique()]

SEASON RANGE:



['2015/2016',
 '2018/2019',
 '2017/2018',
 '2016/2017',
 '2014/2015',
 '2013/2014',
 '2012/2013',
 '2011/2012',
 '2010/2011',
 '2009/2010',
 '2008/2009',
 '2006/2007',
 '2004/2005',
 '2003/2004',
 '1999/2000',
 '1972/1973',
 '1971/1972',
 '1970/1971',
 '2020/2021',
 '2019/2020',
 '2007/2008',
 '2005/2006',
 '1973/1974',
 '1986/1987']

In [50]:
last_5_seasons = ['2020/2021', '2019/2020', '2018/2019', '2017/2018', '2016/2017']
competitions_f2 = competitions_f1.loc[competitions_f1.season_name.isin(last_5_seasons)]

In [51]:
print(f"FILTERED SEASONS:\n")
[season for season in competitions_f2.season_name.unique()]

FILTERED SEASONS:



['2018/2019', '2017/2018', '2016/2017', '2020/2021', '2019/2020']

In [52]:
competitions_f2.shape

(8, 12)

We only have 8 rows left, which means that not all competitions have recent 5 year's data available.

Let's see which competitions have latest data.

In [53]:
print(f"COMPETITIONS THAT HAVE LAST 5 YEARS' DATA:\n")
[c for c in competitions_f2.competition_name.unique()]

COMPETITIONS THAT HAVE LAST 5 YEARS' DATA:



['Champions League', 'La Liga']

In [54]:
print(f"CHAMPIONS LEAGUE SEASONS:\n")
print(competitions_f2.loc[competitions_f2.competition_name=='Champions League'].season_name)
print(f"LA LIGA SEASONS:\n")
print(competitions_f2.loc[competitions_f2.competition_name=='La Liga'].season_name)

CHAMPIONS LEAGUE SEASONS:

1    2018/2019
2    2017/2018
3    2016/2017
Name: season_name, dtype: object
LA LIGA SEASONS:

35    2020/2021
36    2019/2020
37    2018/2019
38    2017/2018
39    2016/2017
Name: season_name, dtype: object


# Matches

The matches folder is categorized by competition ids, containing json files for a full list of matches.

In [55]:
print(f"Competition ids from filtered competitions dataframe:\n")
[i for i in competitions_f2.competition_id.unique()]


Competition ids from filtered competitions dataframe:



[16, 11]

In [56]:
selected_competitions = ['16','11']
for c in selected_competitions:
    root = os.path.join(MATCHES, c)
    filenames = os.listdir(root)
    output = os.path.join(MATCHES, f'{c}_all.json')
    merge_JsonFiles(
        root=root,
        input_files=filenames,
        output_file=output
    )

In [57]:
merge_JsonFiles(
    root=MATCHES,
    input_files=[file for file in os.listdir(MATCHES) if file.endswith('.json')],
    output_file=os.path.join(MATCHES, 'matches_final.json')
)

In [58]:
matches_df = pd.read_json(
    path_or_buf=os.path.join(MATCHES,'matches_final.json')
)

In [59]:
matches_df.head(2)

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,match_status_360,last_updated,last_updated_360,metadata,match_week,competition_stage,stadium,referee
0,9880,2018-04-14,16:15:00.000,"{'competition_id': 11, 'country_name': 'Spain'...","{'season_id': 1, 'season_name': '2017/2018'}","{'home_team_id': 217, 'home_team_name': 'Barce...","{'away_team_id': 207, 'away_team_name': 'Valen...",2,1,available,scheduled,2023-02-08T17:23:53.901920,2021-06-13T16:17:31.694,"{'data_version': '1.1.0', 'shot_fidelity_versi...",32,"{'id': 1, 'name': 'Regular Season'}","{'id': 342, 'name': 'Spotify Camp Nou', 'count...","{'id': 2728, 'name': 'Carlos del Cerro Grande'..."
1,9912,2018-04-29,20:45:00.000,"{'competition_id': 11, 'country_name': 'Spain'...","{'season_id': 1, 'season_name': '2017/2018'}","{'home_team_id': 219, 'home_team_name': 'RC De...","{'away_team_id': 217, 'away_team_name': 'Barce...",2,4,available,scheduled,2022-12-05T14:42:44.641092,2021-06-13T16:17:31.694,"{'data_version': '1.1.0', 'shot_fidelity_versi...",35,"{'id': 1, 'name': 'Regular Season'}","{'id': 4658, 'name': 'Estadio Abanca-Riazor', ...","{'id': 2602, 'name': 'Ricardo De Burgos Bengoe..."


In [60]:
matches_df.season.iloc[0]

{'season_id': 1, 'season_name': '2017/2018'}

In [61]:
print(f"UNIQUE MATCH STATUS:\n")
[s for s in matches_df.match_status.unique()]

UNIQUE MATCH STATUS:



['available']

In [62]:
matches_df.drop(columns=['kick_off','match_status', 'match_status_360', 'last_updated', 'last_updated_360', 'metadata', 'stadium', 'referee'], inplace=True)

In [63]:
matches_df.shape

(5361, 10)

In [64]:
matches_df.head(1)

,match_id,match_date,competition,season,home_team,away_team,home_score,away_score,match_week,competition_stage
0,9880,2018-04-14,"{'competition_id': 11, 'country_name': 'Spain'...","{'season_id': 1, 'season_name': '2017/2018'}","{'home_team_id': 217, 'home_team_name': 'Barce...","{'away_team_id': 207, 'away_team_name': 'Valen...",2,1,32,"{'id': 1, 'name': 'Regular Season'}"


In [65]:
matches_df['competition_id'] = matches_df['competition'].str['competition_id']

In [66]:
matches_df['season_id'] = matches_df['season'].str['season_id']

In [67]:
matches_df.head(1)

,match_id,match_date,competition,season,home_team,away_team,home_score,away_score,match_week,competition_stage,competition_id,season_id
0,9880,2018-04-14,"{'competition_id': 11, 'country_name': 'Spain'...","{'season_id': 1, 'season_name': '2017/2018'}","{'home_team_id': 217, 'home_team_name': 'Barce...","{'away_team_id': 207, 'away_team_name': 'Valen...",2,1,32,"{'id': 1, 'name': 'Regular Season'}",11,1


# Lineups

In [109]:
# Get all match ids from matches_df
match_ids = matches_df.match_id.unique()

In [ ]:
match_ids

In [106]:
lineups_dict = {}
for l in tqdm(os.listdir(LINEUPS)):
    with open(os.path.join(LINEUPS,l), 'r', encoding="utf8") as l_file:
        lineups_dict[l] = json.load(l_file)

  0%|          | 0/3199 [00:00<?, ?it/s]

100%|██████████| 3199/3199 [00:02<00:00, 1134.25it/s]


In [138]:
merge_JsonFiles_with_lookup(
    lookup_ids=match_ids,
    output_file=os.path.join(LINEUPS,'lineups_final.json'),
    lookup_dict = lineups_dict
)

In [68]:
lineups_df = pd.read_json(
    path_or_buf=os.path.join(LINEUPS,'lineups_final.json')
)

In [69]:
lineups_df.head()

,team_id,team_name,lineup,match_id
0,217,Barcelona,"[{'player_id': 3501, 'player_name': 'Philippe ...",9880
1,207,Valencia,"[{'player_id': 4367, 'player_name': 'Gonçalo M...",9880
2,217,Barcelona,"[{'player_id': 3501, 'player_name': 'Philippe ...",9912
3,219,RC Deportivo La Coruña,"[{'player_id': 5537, 'player_name': 'Fabian Lu...",9912
4,217,Barcelona,"[{'player_id': 3501, 'player_name': 'Philippe ...",9924


In [70]:
lineups_df['match_id'].isnull().sum()

0

# Events

In [73]:
match_ids = matches_df.match_id.unique()

In [74]:
final_json = []

for match_id in tqdm(match_ids):
    with open(os.path.join(EVENTS,f'{match_id}.json'), 'r', encoding="utf8") as f:
        loaded_json = json.load(f)
        for i in range(len(loaded_json)):
            if loaded_json[i]['type']['name'] == 'Shot':
                loaded_json[i]['match_id'] = match_id
                final_json.extend([loaded_json[i]])


  0%|          | 0/885 [00:00<?, ?it/s]

100%|██████████| 885/885 [01:31<00:00,  9.62it/s]


In [75]:
len(final_json)

21777

In [76]:
events_df = pd.DataFrame.from_dict(
    final_json
)

In [210]:
events_df

,id,index,period,timestamp,minute,second,type,possession,possession_team,play_pattern,...,player,position,location,duration,related_events,shot,match_id,under_pressure,out,off_camera
0,2258c11c-0916-4976-bfde-a66e89ffeb8d,191,1,00:03:31.114,3,31,"{'id': 16, 'name': 'Shot'}",10,"{'id': 207, 'name': 'Valencia'}","{'id': 4, 'name': 'From Throw In'}",...,"{'id': 4367, 'name': 'Gonçalo Manuel Ganchinho...","{'id': 16, 'name': 'Left Midfield'}","[101.0, 21.3]",1.020612,[12f35fe0-b936-439e-8087-0777a7dc8a38],"{'statsbomb_xg': 0.008752456, 'end_location': ...",9880,NaN,NaN,NaN
1,f0be7ddf-1e01-4315-aee1-8d5fda52a739,383,1,00:07:02.539,7,2,"{'id': 16, 'name': 'Shot'}",14,"{'id': 207, 'name': 'Valencia'}","{'id': 3, 'name': 'From Free Kick'}",...,"{'id': 6594, 'name': 'Santiago Mina Lorenzo'}","{'id': 24, 'name': 'Left Center Forward'}","[103.9, 28.4]",0.127392,"[0638e39d-d8b2-4364-805e-109a33238e0f, 55518d2...","{'statsbomb_xg': 0.05391455, 'end_location': [...",9880,NaN,NaN,NaN
2,bb173d27-76c4-4044-b883-8bd6f8450cf2,654,1,00:13:21.294,13,21,"{'id': 16, 'name': 'Shot'}",25,"{'id': 217, 'name': 'Barcelona'}","{'id': 4, 'name': 'From Throw In'}",...,"{'id': 5211, 'name': 'Jordi Alba Ramos'}","{'id': 6, 'name': 'Left Back'}","[112.5, 26.0]",0.263821,[6c0ad8df-aad9-4e24-aaa9-3a27d8736dfd],"{'statsbomb_xg': 0.08084221, 'end_location': [...",9880,NaN,NaN,NaN
3,7b02c827-3580-4149-9912-216ce2bd544f,700,1,00:14:07.045,14,7,"{'id': 16, 'name': 'Shot'}",26,"{'id': 217, 'name': 'Barcelona'}","{'id': 4, 'name': 'From Throw In'}",...,"{'id': 5246, 'name': 'Luis Alberto Suárez Díaz'}","{'id': 24, 'name': 'Left Center Forward'}","[113.2, 46.0]",0.336449,[ac01843e-3a1a-41de-8090-5cd3e311806f],"{'statsbomb_xg': 0.45647225, 'end_location': [...",9880,NaN,NaN,NaN
4,3b15d864-8738-4cb1-8275-f94277570223,837,1,00:17:27.953,17,27,"{'id': 16, 'name': 'Shot'}",31,"{'id': 207, 'name': 'Valencia'}","{'id': 1, 'name': 'Regular Play'}",...,"{'id': 4367, 'name': 'Gonçalo Manuel Ganchinho...","{'id': 16, 'name': 'Left Midfield'}","[92.8, 32.9]",0.155997,"[2a4a1d74-f213-4ac3-b223-12b8507019c4, e29e5ae...","{'statsbomb_xg': 0.025283428, 'end_location': ...",9880,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21772,62130924-f82f-4772-a4ef-a11fecaa51c2,2758,2,00:38:23.735,83,23,"{'id': 16, 'name': 'Shot'}",214,"{'id': 806, 'name': 'Ajax'}","{'id': 3, 'name': 'From Free Kick'}",...,"{'id': 39723, 'name': 'Sjaak Swart'}","{'id': 17, 'name': 'Right Wing'}","[109.0, 47.1]",1.045860,[1187d7e7-bd66-45cb-9e75-734a39195f2b],"{'statsbomb_xg': 0.03480147, 'end_location': [...",3750235,NaN,NaN,NaN
21773,478faba8-6898-4d23-a74c-fa23672e5b50,2894,2,00:41:35.267,86,35,"{'id': 16, 'name': 'Shot'}",224,"{'id': 806, 'name': 'Ajax'}","{'id': 1, 'name': 'Regular Play'}",...,"{'id': 39724, 'name': 'Gerrie Mühren'}","{'id': 15, 'name': 'Left Center Midfield'}","[98.0, 37.2]",0.950842,[1c222851-880c-45a9-ad9f-6f027c63c479],"{'statsbomb_xg': 0.060509045, 'end_location': ...",3750235,NaN,NaN,NaN
21774,222ec97a-066c-4b4f-9bcf-afb4f842edf2,2935,2,00:42:58.282,87,58,"{'id': 16, 'name': 'Shot'}",227,"{'id': 238, 'name': 'Inter Milan'}","{'id': 1, 'name': 'Regular Play'}",...,"{'id': 39717, 'name': 'Roberto Boninsegna'}","{'id': 23, 'name': 'Center Forward'}","[96.1, 36.4]",0.129619,"[1e6a18b8-2b23-45c1-af8f-5290a778a25d, 369d4b0...","{'statsbomb_xg': 0.036656085, 'end_location': ...",3750235,True,NaN,NaN
21775,70094b8a-f6a2-4b9a-8576-36fef59fe44a,2940,2,00:43:49.185,88,49,"{'id': 16, 'name': 'Shot'}",228,"{'id': 238, 'name': 'Inter Milan'}","{'id': 3, 'name': 'From Free Kick'}",...,"{'id': 39725, 'name': 'Gianfranco Bedin'}","{'id': 11, 'name': 'Left Defensive Midfield'}","[96.8, 38.5]",0.873616,[a1a6d18d-a621-467b-9970-13c1f2061ddf],"{'statsbomb_xg': 0.113855265, 'end_location': ...",3750235,NaN,NaN,NaN


# Export all dataframes

All dataframes are exported to pickle file. The reason of choosing pickle over csv is that columns having dictionaries are converted to string if we used csv, Whereas in pickle, the columns containing dictionaries maintain the dict type. Resulting in dict after loading the pickle file again. This is very helpful.

In [77]:
FINAL_DF = os.path.join(ROOT,'final_dataframes')
competitions_f2.to_pickle(os.path.join(FINAL_DF, 'competitions.pkl'))
matches_df.to_pickle(os.path.join(FINAL_DF, 'matches.pkl'))
lineups_df.to_pickle(os.path.join(FINAL_DF, 'lineups.pkl'))
events_df.to_pickle(os.path.join(FINAL_DF, 'events.pkl'))

In [214]:
tmp_cmp = pd.read_csv(os.path.join(FINAL_DF,'competitions.csv'))
tmp_cmp

,Unnamed: 0,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,1,16,4,Europe,Champions League,male,False,False,2018/2019,2023-03-07T12:20:48.118250,2021-06-13T16:17:31.694,NaN,2023-03-07T12:20:48.118250
1,2,16,1,Europe,Champions League,male,False,False,2017/2018,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,NaN,2021-01-23T21:55:30.425330
2,3,16,2,Europe,Champions League,male,False,False,2016/2017,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,NaN,2020-07-29T05:00
3,35,11,90,Spain,La Liga,male,False,False,2020/2021,2023-07-26T14:11:01.312143,2023-07-26T14:15:15.217027,2023-07-26T14:15:15.217027,2023-07-26T14:11:01.312143
4,36,11,42,Spain,La Liga,male,False,False,2019/2020,2023-07-25T00:16:32.999467,2021-06-13T16:17:31.694,NaN,2023-07-25T00:16:32.999467
5,37,11,4,Spain,La Liga,male,False,False,2018/2019,2023-08-03T02:24:43.761907,2021-07-09T14:53:22.103024,NaN,2023-08-03T02:24:43.761907
6,38,11,1,Spain,La Liga,male,False,False,2017/2018,2023-07-24T13:03:48.574627,2021-06-13T16:17:31.694,NaN,2023-07-24T13:03:48.574627
7,39,11,2,Spain,La Liga,male,False,False,2016/2017,2023-07-25T00:14:55.260536,2021-06-13T16:17:31.694,NaN,2023-07-25T00:14:55.260536


# Utility Functions

In [18]:
# Taken a part from: https://stackoverflow.com/questions/57422734/how-to-merge-multiple-json-files-into-one-file-in-python

def merge_JsonFiles(root, input_files, output_file):
    result = list()
    for f1 in input_files:
        with open(os.path.join(root,f1), 'r', encoding="utf8") as infile:
            result.extend(json.load(infile))

    with open(os.path.join(output_file), 'w', encoding="utf8") as outfile:
        json.dump(result, outfile)

In [137]:
def merge_JsonFiles_with_lookup(lookup_ids, output_file, lookup_dict):
    result = list()
    for id in lookup_ids:
        for team in lookup_dict.get(f'{id}.json'):
            team['match_id'] = id
        result.extend(lookup_dict.get(f'{id}.json',''))

    with open(os.path.join(output_file), 'w', encoding="utf8") as outfile:
        json.dump(result, outfile, cls=NumpyEncoder)

# Ruff Work

In [149]:
# Merging match_id in lineups json...
tmp_result = list()
tmp_result.extend(lineups_dict['9880.json'])
for team in lineups_dict['9880.json']:
    team['match_id'] = '9880'
lineups_dict['9880.json']

[{'team_id': 217,
  'team_name': 'Barcelona',
  'lineup': [{'player_id': 3501,
    'player_name': 'Philippe Coutinho Correia',
    'player_nickname': 'Philippe Coutinho',
    'jersey_number': 14,
    'country': {'id': 31, 'name': 'Brazil'},
    'cards': [],
    'positions': [{'position_id': 12,
      'position': 'Right Midfield',
      'from': '00:00',
      'to': '78:15',
      'from_period': 1,
      'to_period': 2,
      'start_reason': 'Starting XI',
      'end_reason': 'Substitution - Off (Tactical)'}]},
   {'player_id': 5203,
    'player_name': 'Sergio Busquets i Burgos',
    'player_nickname': 'Sergio Busquets',
    'jersey_number': 5,
    'country': {'id': 214, 'name': 'Spain'},
    'cards': [],
    'positions': [{'position_id': 9,
      'position': 'Right Defensive Midfield',
      'from': '00:00',
      'to': None,
      'from_period': 1,
      'to_period': None,
      'start_reason': 'Starting XI',
      'end_reason': 'Final Whistle'}]},
   {'player_id': 5211,
    'player_na